<a href="https://colab.research.google.com/github/VasireddyNandini/predictive-analysis/blob/course_recommend/course%20recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# ---------------------------
# STEP 1: Existing student & course data
# ---------------------------
students = pd.DataFrame([
    {'student_id': 1, 'interests': 'AI,ML,Python', 'focus': 'research', 'performance': 8.5, 'courses_taken': ['C1']},
    {'student_id': 2, 'interests': 'Systems,Networking,C', 'focus': 'industry', 'performance': 7.0, 'courses_taken': ['C2']},
    {'student_id': 3, 'interests': 'AI,Deep Learning,Maths', 'focus': 'research', 'performance': 9.0, 'courses_taken': ['C1', 'C3']},
])

courses = pd.DataFrame([
    {'course_id': 'C1', 'title': 'Intro to AI', 'tags': 'AI,Python', 'track': 'research'},
    {'course_id': 'C2', 'title': 'Computer Networks', 'tags': 'Networking,Systems', 'track': 'industry'},
    {'course_id': 'C3', 'title': 'Deep Learning', 'tags': 'AI,Deep Learning', 'track': 'research'},
    {'course_id': 'C4', 'title': 'Operating Systems', 'tags': 'Systems,C', 'track': 'industry'},
    {'course_id': 'C5', 'title': 'ML for Engineers', 'tags': 'ML,Python,Maths', 'track': 'research'},
])

# ---------------------------
# STEP 2: Input student (new)
# ---------------------------
input_student = {
    'student_id': 99,
    'interests': 'AI,ML,Maths',
    'focus': 'research',
    'performance': 8.8,
    'courses_taken': ['C1']
}

# ---------------------------
# STEP 3: Add input to existing data temporarily
# ---------------------------
# Use pd.concat instead of append
temp_students = pd.concat([students, pd.DataFrame([input_student])], ignore_index=True)


# TF-IDF vector for interests
vectorizer = TfidfVectorizer()
interest_matrix = vectorizer.fit_transform(temp_students['interests'])

# Compute similarity matrix
similarities = cosine_similarity(interest_matrix)

# Get similarity values of input student to others
input_index = temp_students[temp_students['student_id'] == 99].index[0]
temp_students['similarity'] = similarities[input_index]

# Filter top similar students (not the input itself)
similar_students = temp_students[(temp_students['student_id'] != 99) & (temp_students['similarity'] > 0.3)]

# ---------------------------
# STEP 4: Recommend courses from similar students
# ---------------------------
recommended_courses = []

for _, sim_student in similar_students.iterrows():
    for course_id in sim_student['courses_taken']:
        course_info = courses[courses['course_id'] == course_id].iloc[0]
        if course_info['track'] == input_student['focus']:
            if course_id not in input_student['courses_taken']:
                recommended_courses.append((course_info['title'], course_info['tags']))

# Remove duplicates
recommended_titles = list(set([title for title, tags in recommended_courses]))

# ---------------------------
# STEP 5: Content filter for interests (optional boost)
# ---------------------------
for _, course in courses.iterrows():
    if input_student['focus'] == course['track'] and course['course_id'] not in input_student['courses_taken']:
        if any(interest.strip() in course['tags'] for interest in input_student['interests'].split(',')):
            recommended_titles.append(course['title'])

# Final deduped recommendations
final_recommendations = list(set(recommended_titles))

print("\n🎓 Recommended Courses for the Student:")
for course in final_recommendations:
    print("✅", course)


🎓 Recommended Courses for the Student:
✅ ML for Engineers
✅ Deep Learning
